# Imports

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Data 
## Get data

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

# remove entries with missing values
dataset = raw_dataset.dropna()
# from sklearn import preprocessing
# normalized_features = preprocessing.StandardScaler().fit_transform(dataset)
# dataset = pd.DataFrame(data=normalized_features, columns=column_names) 

## Inspect Data

In [ ]:
print('Dataset shape:')
print(dataset.shape)

print('Tail:')
print(dataset.tail())

print('Statistics:')
print(dataset.describe().transpose())

sns.pairplot(dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')
plt.show()

## Split dataset

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


# Simple Linear Regression
predict MPG (y, dependent variable) using Weight (x, independent variable) using closed-form solution y = theta_0 + theta_1 * x - we want to find theta_0 and theta_1 parameters that minimize the prediction error

We can calculate the error using MSE metric: 
MSE = SUM (from i=1 to n) (actual_output - predicted_output) ** 2

In [ ]:
# get the columns
y_train = train_dataset['MPG'].to_numpy()
x_train = train_dataset['Weight'].to_numpy()

y_test = test_dataset['MPG'].to_numpy()
x_test = test_dataset['Weight'].to_numpy()


## Closed-form solution

In [ ]:
X = np.c_[np.ones((len(x_train), 1)), x_train]
theta_best = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y_train)
print('Theta:', theta_best)

## Calculate error

In [ ]:
def calculate_mse(_theta, _x, _y):
    m = len(_x)
    y_p = _x.dot(_theta)
    return np.sum((y_p - _y) ** 2) / m

X_test = np.c_[np.ones((len(x_test), 1)), x_test]
X_train = np.c_[np.ones((len(x_train), 1)), x_train]

print('MSE for train set:', calculate_mse(theta_best, X_train, y_train))
print('MSE for test set:', calculate_mse(theta_best, X_test, y_test))


## Plot the regression line

In [ ]:
x = np.linspace(min(x_test), max(x_test), 100)
y = float(theta_best[0]) + float(theta_best[1]) * x
plt.plot(x, y)
plt.scatter(x_test, y_test)
plt.xlabel('Weight')
plt.ylabel('MPG')
plt.show()

## Standardization

In [ ]:
x_standard_deviation = np.std(x_train)
x_average = np.average(x_train)
y_standard_deviation = np.std(y_train)
y_average = np.average(y_train)

x_train_standardized = (x_train - x_average) / x_standard_deviation
y_train_standardized = (y_train - y_average) / y_standard_deviation
x_test_standardized = (x_test - x_average) / x_standard_deviation
y_test_standardized = (y_test - y_average) / y_standard_deviation

X_train_standardized = np.c_[np.ones((len(x_train_standardized), 1)), x_train_standardized]
X_test_standardized = np.c_[np.ones((len(x_test_standardized), 1)), x_test_standardized]
Y_train_standardized = y_train_standardized.reshape(-1, 1)
Y_test_standardized = y_test_standardized.reshape(-1, 1)


## Calculate theta using Batch Gradient Descent

In [ ]:
#theta = theta_best.reshape(-1, 1)
theta = np.random.randn(2, 1)
eta = 0.0001

def calculate_mse_gradient(_theta, _x, _y):
    m = len(_x)
    return 2/m * (_x.T.dot(_x.dot(_theta) - _y))

def calculate_theta_using_batch_gradient_descent(_theta, _x, _y, _eta):
    previous_mse = None
    while True:
        gradients = calculate_mse_gradient(_theta, _x, _y)
        _theta = _theta - _eta * gradients
        current_mse = calculate_mse(_theta, _x, _y)
        if previous_mse is not None and current_mse == previous_mse:
            break
        previous_mse = current_mse
    return _theta

theta = calculate_theta_using_batch_gradient_descent(theta, X_train_standardized, Y_train_standardized, eta)

In [ ]:
scaled_theta = theta.copy()
scaled_theta[1] = scaled_theta[1] * y_standard_deviation / x_standard_deviation
scaled_theta[0] = y_average - scaled_theta[1] * x_average
scaled_theta = scaled_theta.reshape(-1)

print('Theta:', scaled_theta)
print('Best theta:', theta_best)


## Calculate error

In [ ]:
print('MSE for train set:', calculate_mse(scaled_theta, X_train, y_train))
print('MSE for test set:', calculate_mse(scaled_theta, X_test, y_test))
print('MSE for train set (closed-form solution):', calculate_mse(theta_best, X_train, y_train))
print('MSE for test set (closed-form solution):', calculate_mse(theta_best, X_test, y_test))

In [ ]:
x = np.linspace(min(x_test), max(x_test), 100)
y = float(scaled_theta[0]) + float(scaled_theta[1]) * x
plt.plot(x, y)
plt.scatter(x_test, y_test)
plt.xlabel('Weight')
plt.ylabel('MPG')
plt.show()